In [ ]:
import multiprocessing as mp
import os
import random

import numpy as np
import pandas as pd
import perturb_ecg
import tools
import tqdm
import vcg
from matplotlib import pyplot as plt
from mpl_toolkits import mplot3d

In [ ]:
print(mp.cpu_count())

In [ ]:
import seaborn as sns

sns.set()


SMALL_SIZE = 15
MEDIUM_SIZE = 20
BIGGER_SIZE = 25

plt.rc("font", size=SMALL_SIZE)  # controls default text sizes
plt.rc("axes", titlesize=SMALL_SIZE)  # fontsize of the axes title
plt.rc("axes", labelsize=MEDIUM_SIZE)  # fontsize of the x and y labels
plt.rc("xtick", labelsize=SMALL_SIZE)  # fontsize of the tick labels
plt.rc("ytick", labelsize=SMALL_SIZE)  # fontsize of the tick labels
plt.rc("legend", fontsize=SMALL_SIZE)  # legend fontsize
plt.rc("figure", titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [ ]:
import copy

In [ ]:
dataset = "ecg_large"
if not os.path.exists(dataset):
    os.makedirs(f"./{dataset}/train")
    os.makedirs(f"./{dataset}/val/train")
    os.makedirs(f"./{dataset}/val/test")

fs = 496  # frequency
duration = 100  # seconds to solve
save_duration = 10  # seconds to save
dur = save_duration * fs  # number of samples to save
upper_hr = 220
lower_hr = 20
hr_step = 0.1
draw_num = 2
types = 8


keys = pd.DataFrame(
    np.random.randint(
        0, (duration - save_duration) * fs, size=(int((upper_hr - lower_hr) / hr_step), draw_num * types)
    )
)
keys.columns = [
    "ind0",
    "ind1",
    "ind2",
    "ind3",
    "ind4",
    "ind5",
    "ind6",
    "ind7",
    "ind8",
    "ind9",
    "ind10",
    "ind11",
    "ind12",
    "ind13",
    "ind14",
    "ind15",
]

keys["hr"] = range(lower_hr * 10, upper_hr * 10)
keysc = keys.sample(frac=0.2)
keysc["train"] = int(0)
val_keys = keysc.sample(frac=0.2)
val_keys["val_train"] = int(0)
key = keysc.merge(val_keys, how="outer")
df_key = keys.merge(key, how="outer")
df_key = df_key.fillna(1)
df_key["val_train"].value_counts()
df_key["train"] = df_key["train"].astype(int)
df_key["val_train"] = df_key["val_train"].astype(int)
df_key

In [ ]:
def amp_qrs(vcg_ode_original, scale=1.5):

    vcg_ode = copy.deepcopy(vcg_ode_original)

    b_x = vcg_ode.b_x
    b_y = vcg_ode.b_y
    b_z = vcg_ode.b_z

    alpha_x = vcg_ode.alpha_x
    alpha_y = vcg_ode.alpha_y
    alpha_z = vcg_ode.alpha_z

    # make bigger
    alpha_x[3] *= scale
    alpha_x[4] *= scale
    alpha_x[5] *= scale

    alpha_y[3] *= scale
    alpha_y[4] *= scale
    alpha_y[5] *= scale

    alpha_z[3] *= scale
    alpha_z[4] *= scale
    alpha_z[5] *= scale

    vcg_ode.b_x = b_x
    vcg_ode.b_y = b_y
    vcg_ode.b_z = b_z

    vcg_ode.alpha_x = alpha_x
    vcg_ode.alpha_y = alpha_y
    vcg_ode.alpha_z = alpha_z

    return vcg_ode

In [ ]:
def amp_pwave1(vcg_ode_original, scale=25):

    vcg_ode = copy.deepcopy(vcg_ode_original)

    b_x = vcg_ode.b_x
    b_y = vcg_ode.b_y
    b_z = vcg_ode.b_z

    alpha_x = vcg_ode.alpha_x
    alpha_y = vcg_ode.alpha_y
    alpha_z = vcg_ode.alpha_z

    # make bigger
    alpha_x[0] *= scale
    #     alpha_x[1] *= scale

    #     alpha_y[0] *= scale
    #     alpha_y[1] *= scale
    #     alpha_y[2] *= scale

    #     alpha_z[0] *= scale
    #     alpha_z[1] *= scale
    #     alpha_z[2] *= scale

    vcg_ode.b_x = b_x
    vcg_ode.b_y = b_y
    vcg_ode.b_z = b_z

    vcg_ode.alpha_x = alpha_x
    vcg_ode.alpha_y = alpha_y
    vcg_ode.alpha_z = alpha_z

    return vcg_ode

In [ ]:
def amp_pwave2(vcg_ode_original, scale=25):

    vcg_ode = copy.deepcopy(vcg_ode_original)

    b_x = vcg_ode.b_x
    b_y = vcg_ode.b_y
    b_z = vcg_ode.b_z

    alpha_x = vcg_ode.alpha_x
    alpha_y = vcg_ode.alpha_y
    alpha_z = vcg_ode.alpha_z

    # make bigger
    alpha_x[0] *= scale
    #     alpha_x[1] *= scale

    alpha_y[0] *= scale
    #     alpha_y[1] *= scale
    #     alpha_y[2] *= scale

    alpha_z[0] *= scale
    #     alpha_z[1] *= scale
    #     alpha_z[2] *= scale

    vcg_ode.b_x = b_x
    vcg_ode.b_y = b_y
    vcg_ode.b_z = b_z

    vcg_ode.alpha_x = alpha_x
    vcg_ode.alpha_y = alpha_y
    vcg_ode.alpha_z = alpha_z

    return vcg_ode

In [ ]:
def amp_twave(vcg_ode_original, scale=5):

    vcg_ode = copy.deepcopy(vcg_ode_original)

    b_x = vcg_ode.b_x
    b_y = vcg_ode.b_y
    b_z = vcg_ode.b_z

    alpha_x = vcg_ode.alpha_x
    alpha_y = vcg_ode.alpha_y
    alpha_z = vcg_ode.alpha_z

    # make bigger
    alpha_x[8] *= scale
    #     alpha_x[4] *= scale
    #     alpha_x[5] *= scale

    #     alpha_y[8] *= scale
    #     alpha_y[4] *= scale
    #     alpha_y[5] *= scale

    #     alpha_z[8] *= scale
    #     alpha_z[4] *= scale
    #     alpha_z[5] *= scale

    vcg_ode.b_x = b_x
    vcg_ode.b_y = b_y
    vcg_ode.b_z = b_z

    vcg_ode.alpha_x = alpha_x
    vcg_ode.alpha_y = alpha_y
    vcg_ode.alpha_z = alpha_z

    return vcg_ode

In [ ]:
def st_change(vcg_ode_original, scale=6):

    vcg_ode = copy.deepcopy(vcg_ode_original)

    b_x = vcg_ode.b_x
    b_y = vcg_ode.b_y
    b_z = vcg_ode.b_z

    alpha_x = vcg_ode.alpha_x
    alpha_y = vcg_ode.alpha_y
    alpha_z = vcg_ode.alpha_z

    # make bigger
    #     alpha_x[6] *= scale
    #     alpha_x[4] *= scaled
    #     alpha_x[6] *= scale
    alpha_x[6] *= scale

    alpha_y[6] *= scale
    #     alpha_y[7] *= scale
    #     alpha_y[5] *= scale

    #     alpha_z[8] *= scale
    #     alpha_z[4] *= scale
    #     alpha_z[6] *= scale

    vcg_ode.b_x = b_x
    vcg_ode.b_y = b_y
    vcg_ode.b_z = b_z

    vcg_ode.alpha_x = alpha_x
    vcg_ode.alpha_y = alpha_y
    vcg_ode.alpha_z = alpha_z

    return vcg_ode

In [ ]:
def make_ecgs(row):

    # find folder for the ecgs to go in
    if row[17] == 1:
        path = f"./{dataset}/train"
    elif row[18] == 1:
        path = f"./{dataset}/val/train"
    else:
        path = f"./{dataset}/val/test/"

    for i in range(0, 8):
        if not os.path.isdir(f"{path}/{row[16]}_{i}"):
            os.mkdirs(f"{path}/{row[16]}_{i}")
    # make the base ode solves
    vcg_ode = vcg.VCG(row[16] / 10)
    # get actual arrays of values and save
    # orig
    _, y0 = tools.solve_vcg_object(vcg_ode, duration=duration, fs=fs)
    pd.DataFrame(y0[row[0] : row[0] + save_duration * fs, :].T).to_csv(
        f"{path}/{row[16]}_0/{row[16]}_0_{row[0]}.csv", header=False, index=False
    )
    pd.DataFrame(y0[row[1] : row[1] + save_duration * fs, :].T).to_csv(
        f"{path}/{row[16]}_0/{row[16]}_0_{row[1]}.csv", header=False, index=False
    )
    # qt_elong
    vcg_ode2 = perturb_ecg.qt_elongation(vcg_ode, ms_forward=np.random.randint(50, 250))
    _, y1 = tools.solve_vcg_object(vcg_ode2, duration=duration, fs=fs)
    pd.DataFrame(y1[row[2] : row[2] + save_duration * fs, :].T).to_csv(
        f"{path}/{row[16]}_1/{row[16]}_1_{row[2]}.csv", header=False, index=False
    )
    pd.DataFrame(y1[row[3] : row[3] + save_duration * fs, :].T).to_csv(
        f"{path}/{row[16]}_1/{row[16]}_1_{row[3]}.csv", header=False, index=False
    )
    # wide_qrs
    vcg_ode3 = perturb_ecg.wide_qrs(
        vcg_ode, percent_widened=np.random.randint(200, 1000), scaledown=(np.random.randint(10, 100) / 100)
    )
    _, y2 = tools.solve_vcg_object(vcg_ode3, duration=duration, fs=fs)
    pd.DataFrame(y2[row[4] : row[4] + save_duration * fs, :].T).to_csv(
        f"{path}/{row[16]}_2/{row[16]}_2_{row[4]}.csv", header=False, index=False
    )
    pd.DataFrame(y2[row[5] : row[5] + save_duration * fs, :].T).to_csv(
        f"{path}/{row[16]}_2/{row[16]}_2_{row[5]}.csv", header=False, index=False
    )
    # amp_qrs
    vcg_ode4 = amp_qrs(vcg_ode, scale=(np.random.randint(8, 30) / 10))
    _, y3 = tools.solve_vcg_object(vcg_ode4, duration=duration, fs=fs)
    pd.DataFrame(y3[row[6] : row[6] + save_duration * fs, :].T).to_csv(
        f"{path}/{row[16]}_3/{row[16]}_3_{row[6]}.csv", header=False, index=False
    )
    pd.DataFrame(y3[row[7] : row[7] + save_duration * fs, :].T).to_csv(
        f"{path}/{row[16]}_3/{row[16]}_3_{row[7]}.csv", header=False, index=False
    )
    # amp_pwave1
    vcg_ode5 = amp_pwave1(vcg_ode, scale=np.random.randint(2, 50))
    _, y4 = tools.solve_vcg_object(vcg_ode5, duration=duration, fs=fs)
    pd.DataFrame(y4[row[8] : row[8] + save_duration * fs, :].T).to_csv(
        f"{path}/{row[16]}_4/{row[16]}_4_{row[8]}.csv", header=False, index=False
    )
    pd.DataFrame(y4[row[9] : row[9] + save_duration * fs, :].T).to_csv(
        f"{path}/{row[16]}_4/{row[16]}_4_{row[9]}.csv", header=False, index=False
    )
    # amp_pwave2
    vcg_ode6 = amp_pwave2(vcg_ode, scale=np.random.randint(2, 50))
    _, y5 = tools.solve_vcg_object(vcg_ode6, duration=duration, fs=fs)
    pd.DataFrame(y5[row[10] : row[10] + save_duration * fs, :].T).to_csv(
        f"{path}/{row[16]}_5/{row[16]}_5_{row[10]}.csv", header=False, index=False
    )
    pd.DataFrame(y5[row[11] : row[11] + save_duration * fs, :].T).to_csv(
        f"{path}/{row[16]}_5/{row[16]}_5_{row[11]}.csv", header=False, index=False
    )
    # amp_twave
    vcg_ode7 = amp_twave(vcg_ode, scale=np.random.randint(2, 10))
    _, y6 = tools.solve_vcg_object(vcg_ode7, duration=duration, fs=fs)
    pd.DataFrame(y6[row[12] : row[12] + save_duration * fs, :].T).to_csv(
        f"{path}/{row[16]}_6/{row[16]}_6_{row[12]}.csv", header=False, index=False
    )
    pd.DataFrame(y6[row[13] : row[13] + save_duration * fs, :].T).to_csv(
        f"{path}/{row[16]}_6/{row[16]}_6_{row[13]}.csv", header=False, index=False
    )
    # st_change
    vcg_ode8 = st_change(vcg_ode, scale=np.random.randint(2, 10))
    _, y7 = tools.solve_vcg_object(vcg_ode8, duration=duration, fs=fs)
    pd.DataFrame(y7[row[14] : row[14] + save_duration * fs, :].T).to_csv(
        f"{path}/{row[16]}_7/{row[16]}_7_{row[14]}.csv", header=False, index=False
    )
    pd.DataFrame(y7[row[15] : row[15] + save_duration * fs, :].T).to_csv(
        f"{path}/{row[16]}_7/{row[16]}_7_{row[15]}.csv", header=False, index=False
    )

    del vcg_ode
    del vcg_ode2
    del vcg_ode3
    del vcg_ode4
    del vcg_ode5
    del vcg_ode6
    del vcg_ode7
    del vcg_ode8

    print(row[16])
    return [row[16], path]

In [ ]:
# convert keys to list for pool
key_list = df_key.values.tolist()

In [ ]:
pool = mp.Pool(mp.cpu_count())
results = pool.map(make_ecgs, [row for row in key_list])
print("Done!")
pool.close()

In [ ]:
df_key.to_csv(f"./{dataset}/ecg_keys.csv")